In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
import requests
from dotenv import load_dotenv

d:\LangChain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

In [3]:
search_tool = DuckDuckGoSearchRun()

In [4]:
@tool
def get_weather_data(city : str) -> str:
    """
    this function fetchs the current weather data for a given city"""
    
    url = f"pass the weather api={city}"
    
    response = requests.get(url)
    
    return response.json

In [5]:
result = search_tool.invoke("today odisha")

In [5]:
print(result)

1 day ago · Get daily Top Stories on Opinions, Crime, Education, Entertainment, Weather, Sports, Business, Trending, Viral, and LIVE Video News Coverage from leading columnists only at Odisha TV. 20 hours ago · Cyclone Montha LIVE: Storm to hit Andhra coast tonight. IMD issues red alert for Andhra, Odisha and Tamil Nadu. Track real-time updates on landfall timing, rainfall intensity, evacuation efforts, and weather warnings. Read full articles, watch videos, browse thousands of titles and more on the " Odisha " topic with Google News. 2 days ago · Cyclone Montha over the Bay of Bengal is set to intensify into a severe cyclonic storm by October 28 morning, likely making landfall near Kakinada. Andhra Pradesh has been put on alert while Odisha , Telangana, and Chennai are also bracing for strong winds and rainfall as the system moves north-northwestward. 17 hours ago · Cyclone Montha Live: 100 kmph Winds Hit Andhra Coast as Landfall Begins; Vehicle Curfew in Place — Track Cyclone’s Live 

In [6]:
llm = ChatGoogleGenerativeAI(model = "gemini-2.5-flash")

In [7]:
result = llm.invoke("hey i am suvendu")

In [8]:
print(result)

content='Hi Suvendu! Nice to meet you. How can I help you today?' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run--111130b0-aebb-4a9d-85a6-0403aa72f71d-0' usage_metadata={'input_tokens': 6, 'output_tokens': 17, 'total_tokens': 58, 'input_token_details': {'cache_read': 0}}


In [9]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

In [11]:
# step:2 pull the ReAct prompt from langchain hub.

prompt = hub.pull("hwchase17/react")  # pull the standard ReAct agent prompt


"""
react means resoning + action

“It is a predefined prompt function — instead of writing the pattern manually,
 we just call it,
 and it automatically applies the prompt structure.”

 """

'\nreact means resoning + action\n\n“It is a predefined prompt function — instead of writing the pattern manually,\n we just call it,\n and it automatically applies the prompt structure.”\n\n '

In [17]:
print(prompt)

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'} template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'


In [ ]:
# step:3 crete react agent manually with the pulled prompt
agent = create_react_agent(
    llm = llm,
    prompt = prompt,
    tools = [search_tool, get_weather_data]
)

"""create_react_agent() is used to teach the LLM how to work automatically —
that means:

How to think (reasoning step)

What to do next (decide if it needs a tool)

How to do it automatically (call the tool without you telling it manually)"""

"""it means to think what to do"""

'it means to think what to do'

In [ ]:
# step:4 wrap it agentEcecuter
agent_executor = AgentExecutor(
    agent = agent,
    tools = [search_tool, get_weather_data],
    verbose = True,
    handle_parsing_errors = True
)


"""it means after thinking what to do, execute it"""

"""examples :- Suppose we want to find the temperature of Delhi and then multiply it by 10.

First, the agent reasons:
“To get Delhi’s temperature, I need to call an API or use a search tool.”

The agent executor then performs that action (fetches the temperature).

After receiving the result, the agent thinks again:
“Now I need to multiply this temperature by 10.”

It sends that instruction to the executor, which carries out the calculation.

So, the ReAct approach involves a continuous cycle of Reason → Action → Observation → Next
Reasoning, allowing the agent to think and act intelligently step by step."""

'examples :- Suppose we want to find the temperature of Delhi and then multiply it by 10.\n\nFirst, the agent reasons:\n“To get Delhi’s temperature, I need to call an API or use a search tool.”\n\nThe agent executor then performs that action (fetches the temperature).\n\nAfter receiving the result, the agent thinks again:\n“Now I need to multiply this temperature by 10.”\n\nIt sends that instruction to the executor, which carries out the calculation.\n\nSo, the ReAct approach involves a continuous cycle of Reason → Action → Observation → Next\nReasoning, allowing the agent to think and act intelligently step by step.'

| Step                  | Who does it         | What happens                                                                                           |
| --------------------- | ------------------- | ------------------------------------------------------------------------------------------------------ |
| 🧠 **Thought**        | **LLM**             | The LLM reasons: “To answer this, I need to find travel methods.”                                      |
| 🧩 **Format (ReAct)** | **ReAct framework** | The ReAct prompt *tells the LLM* to always show its reasoning as `Thought:` and tool use as `Action:`. |
| 🧰 **Execution**      | **Agent Executor**  | Runs the chosen tool (e.g., search) with the given input.                                              |
| 👀 **Observation**    | **LLM + ReAct**     | The result is fed back, and the LLM continues reasoning.                                               |


In [ ]:
# step 5 : Invoke
response = agent_executor.invoke({"input": "3 way to reach goa from delhi."})
print(response)

"""we use verbose = true. so that we can see that how the agent think and how
agent executor perform """



> Entering new AgentExecutor chain...
Action: duckduckgo_search
Action Input: how to reach goa from delhiHow to reach Goa from Delhi without any money ... However, you can ask whether it is possible/easy/in the culture to hitch-hike on cars, trucks or ... If you want to reach Goa from Delhi , you have many options, like by train, by air or by road. ... is the most convenient way to reach Goa from Delhi ... The best way to reach Goa from Delhi is to take a flight. ... By Train: Direct trains are available that you can avail to reach Goa from Delhi . A tourist destination, Goa can be reached by air, rail or road from Delhi . ... If you have any more questions about the route from Delhi to Goa by ... How to Reach Goa from Delhi ? ... By Air: Direct flights from Delhi to Goa take approximately 2.5 hours, making it the fastest and most convenient ...I now know the final answer
Final Answer: Three ways to reach Goa from Delhi are by air, by train, and by road.

> Finished chain.
{'input': 

In [22]:
response['output']

'Three ways to reach Goa from Delhi are by air, by train, and by road.'